In [1]:
import argparse
from SoccerNet.utils import getListGames
import os
import json
from tqdm import tqdm
import numpy as np
from SoccerNet.Evaluation.ActionSpotting import evaluate
from collections import defaultdict

In [2]:
parser = argparse.ArgumentParser()
# parser.add_argument("--anno_file",default='')
parser.add_argument("--source_test_path",default='/youtu_pedestrian_detection/chengzhilin/program/soccernet/models/MVIT_FULL/outputs_test')#输入test文件夹 
parser.add_argument("--output_test_path",default='/youtu_pedestrian_detection/zhuhe/soccernet/submit/output_full_thred_2')  #筛选后test文件夹

parser.add_argument("--source_challenge_path",default='/youtu_pedestrian_detection/zhuhe/soccernet/submit/challenge_224_all')#输入challenge文件夹 
parser.add_argument("--output_challenge_path", default="/youtu_pedestrian_detection/zhuhe/soccernet/submit/challenge_224_all_thred")  #输出challenge文件夹
# parser.add_argument('--window_start', type=int,   default=10,     help='Size of the chunk (in seconds)' )
# parser.add_argument('--window_end', type=int,   default=50,     help='Size of the chunk (in seconds)' )
# parser.add_argument('--window_gap',  type=int,   default=0.5,     help='Size of the chunk (in seconds)' )

parser.add_argument('--window_size',  type=int,   default=26.5,     help='Size of the chunk (in seconds)' )

parser.add_argument('--thred_start', type=int,   default=0,     help='Size of the chunk (in seconds)' )
parser.add_argument('--thred_end', type=int,   default=0.3,     help='Size of the chunk (in seconds)' )
parser.add_argument('--thred_gap',  type=int,   default=0.1,     help='Size of the chunk (in seconds)' )

args=parser.parse_args([])

In [3]:
os.makedirs(args.output_challenge_path,exist_ok='True')

In [4]:
thred={
    'Ball out of play':0,
    'Throw-in':0,
    'Foul':0,
    'Indirect free-kick':0,
    'Clearance':0,
    'Shots on target':0,
    'Shots off target':0,
    'Corner':0,
    'Substitution':0,
    'Kick-off':0,
    'Direct free-kick':0,
    'Offside':0,
    'Yellow card':0,
    'Goal':0,
    'Penalty':0,
    'Red card':0,
    'Yellow->red card':0,
    'Background':0
}

In [5]:
def process_window(window):
    action_dic={}
    res=[]
    for pre in window:
        if float(pre['confidence']) <  thred[pre['label']]: continue
        
        if pre['label'] not in action_dic:
            action_dic[pre['label']]=pre
        else:
            if float(pre['confidence']) > float(action_dic[pre['label']]['confidence']):
                action_dic[pre['label']]=pre
    for label in action_dic.keys():
        res.append(action_dic[label])
    return res

In [6]:
total_results=defaultdict(list)
window_size=args.window_size

for action in tqdm(thred.keys()):
    thred[action]=args.thred_start
    while thred[action]<=args.thred_end:
        
        for cata in getListGames(split="test"):

            file=os.path.join(args.source_test_path,cata,'results_spotting.json')
            with open(file,'r') as f:
                data=json.load(f)
            pres=data['predictions']
            pres.sort(key=lambda x:(int(x['half']),int(x['position'])))

            json_data = {} 
            total_pre={}
            json_data["UrlLocal"]=data['UrlLocal']
            json_data["predictions"]=[]

            last_half=1
            start,end=0,window_size
            window=[]
            for cur_pre,pre in enumerate(pres):


                if start<= int(pre['position'])/1000.0 <end:
                    window.append(pre)
                else:
                    json_data["predictions"]+=process_window(window)
                    window=[pre]
                    # while end<int(pre['position']):
                    start+=window_size
                    end+=window_size

                    if last_half==1 and int(pre['half'])==2:
                        last_half=2
                        start,end=0,window_size

            json_data["predictions"]+=process_window(window)

            os.makedirs(os.path.join(args.output_test_path,cata),exist_ok='True')
            with open(os.path.join(args.output_test_path,cata,'results_spotting.json'),'w') as f:
                json.dump(json_data,f,indent=2)

        PATH_DATASET='/youtu_pedestrian_detection/junweil/action_datasets/soccernet'
        PATH_PREDICTIONS=args.output_test_path


        from SoccerNet.Evaluation.ActionSpotting import evaluate
        results = evaluate(SoccerNet_path=PATH_DATASET, Predictions_path=PATH_PREDICTIONS,
                           split="test", version=2, prediction_file="results_spotting.json", metric="tight")

        a_map,a_mAP_visible,a_mAP_unshown=results["a_mAP"],results["a_mAP_visible"],results["a_mAP_unshown"]
        # total_results.append([window_size,a_map,a_mAP_visible,a_mAP_unshown])
        total_results[action].append([thred[action],a_map,a_mAP_visible,a_mAP_unshown])

        thred[action]+=args.thred_gap
        
    thred[action]=0

100%|██████████| 100/100 [00:20<00:00,  4.85it/s]

100%|██████████| 5/5 [00:13<00:00,  2.64s/it]

100%|██████████| 100/100 [00:14<00:00,  6.75it/s]

100%|██████████| 5/5 [00:13<00:00,  2.63s/it]

100%|██████████| 100/100 [00:15<00:00,  6.36it/s]

100%|██████████| 100/100 [00:17<00:00,  5.71it/s]

100%|██████████| 5/5 [00:13<00:00,  2.70s/it]

100%|██████████| 100/100 [00:16<00:00,  6.23it/s]

100%|██████████| 5/5 [00:12<00:00,  2.58s/it]

100%|██████████| 100/100 [00:14<00:00,  6.73it/s][A

100%|██████████| 100/100 [00:16<00:00,  6.02it/s]

100%|██████████| 5/5 [00:13<00:00,  2.64s/it]

100%|██████████| 100/100 [00:14<00:00,  7.09it/s]

100%|██████████| 5/5 [00:12<00:00,  2.58s/it]

100%|██████████| 100/100 [00:13<00:00,  7.29it/s]

100%|██████████| 100/100 [00:19<00:00,  5.14it/s]

100%|██████████| 5/5 [00:13<00:00,  2.64s/it]

100%|██████████| 100/100 [00:12<00:00,  7.88it/s]

100%|██████████| 5/5 [00:12<00:00,  2.55s/it]

100%|██████████| 100/100 [00:09<00:00, 10.86it/s]

100%|█████

OSError: [Errno 122] Disk quota exceeded

In [13]:
thred

{'Ball out of play': 0,
 'Throw-in': 0,
 'Foul': 0,
 'Indirect free-kick': 0,
 'Clearance': 0,
 'Shots on target': 0,
 'Shots off target': 0,
 'Corner': 0,
 'Substitution': 0,
 'Kick-off': 0,
 'Direct free-kick': 0,
 'Offside': 0,
 'Yellow card': 0,
 'Goal': 0,
 'Penalty': 0,
 'Red card': 0,
 'Yellow->red card': 0,
 'Background': 0}

In [12]:
total_results

defaultdict(list,
            {'Ball out of play': [[0,
               0.37189007693837783,
               0.45514818894529907,
               0.13240919371158277],
              [0.1,
               0.37189007693837783,
               0.45514818894529907,
               0.13203107310445453],
              [0.2,
               0.3714445878088987,
               0.45514818894529907,
               0.13203107310445453],
              [0.30000000000000004,
               0.3714445878088987,
               0.4546464334098516,
               0.13177963249768399],
              [0.4,
               0.37118143366543005,
               0.4543645009039461,
               0.13177963249768399],
              [0.5,
               0.3706764005180269,
               0.453830260345737,
               0.13125291562481325],
              [0.6,
               0.37004123713303777,
               0.453830260345737,
               0.13107213533454393],
              [0.7,
               0.3681204613319301,

In [111]:
for action in (thred.keys()):
               
    total_results[action].sort(key=lambda x:-x[1])
    thred[action]=total_results[action][0][0]
               


In [112]:
window_size

10.0

In [113]:
total_results

[[10.0, 0.3635886168702596, 0.4452361951226681, 0.13416065650464098],
 [9.0, 0.3628183949734276, 0.4408861857435958, 0.13129837213006507],
 [8.0, 0.3623251507985085, 0.4394945123104102, 0.13513755799699895],
 [9.5, 0.36051810512867954, 0.43839204361662265, 0.13315908195648835],
 [8.5, 0.3597629987587144, 0.440094394295939, 0.13528863884583778],
 [7.0, 0.3574157910817314, 0.4357392284301309, 0.13575448732562678],
 [6.5, 0.3571480587524357, 0.43471564446898003, 0.13501414585936694],
 [7.5, 0.35633529300113514, 0.43441008626314975, 0.13633516850227909],
 [5.5, 0.35045527138076066, 0.4247491026570801, 0.13260608574209778],
 [6.0, 0.3503767726499682, 0.42360994858021006, 0.1333123736889443],
 [5.0, 0.3486040402767273, 0.4253135064499454, 0.1337106279009972],
 [4.0, 0.34435912502347854, 0.41643962807336105, 0.1324311198498365],
 [4.5, 0.3424705960086811, 0.41638542665409883, 0.1315793889648043],
 [3.5, 0.3370436435125208, 0.40952792182902686, 0.13061871362556607],
 [3.0, 0.32788232376722665,

In [9]:
for cata in tqdm(getListGames(split="challenge")):
    
    file=os.path.join(args.source_challenge_path,cata,'results_spotting.json')
    with open(file,'r') as f:
        data=json.load(f)
    pres=data['predictions']
    pres.sort(key=lambda x:(int(x['half']),int(x['position'])))
    
    json_data = {} 
    total_pre={}
    json_data["UrlLocal"]=data['UrlLocal']
    json_data["predictions"]=[]
    
    last_half=1
    start,end=0,window_size
    window=[]
    for cur_pre,pre in enumerate(pres):

        
        if start<= int(pre['position'])/1000.0 <end:
            window.append(pre)
        else:
            json_data["predictions"]+=process_window(window)
            window=[pre]
            start+=window_size
            end+=window_size
            
            if last_half==1 and int(pre['half'])==2:
                last_half=2
                start,end=0,window_size
         
    json_data["predictions"]+=process_window(window)
    
    os.makedirs(os.path.join(args.output_challenge_path,cata),exist_ok='True')
    with open(os.path.join(args.output_challenge_path,cata,'results_spotting.json'),'w') as f:
        json.dump(json_data,f,indent=2)
    
    
